# DABstep Benchmark Qwen2.5-Coder-32B Baseline

This notebook will guide you though submitting a Qwen2.5-Coder-32B baseline to the DABstep leaderboard.

* Live 🤗 Leaderboard: https://huggingface.co/spaces/adyen/DABstep
* Benchmark 🤗 Dataset: https://huggingface.co/datasets/adyen/DABstep
* LLM Agent Framework by 🤗: https://github.com/huggingface/smolagents/tree/main



## Environment Setup

We need to setup:
* **HuggingFace Token:** In order to make free API calls to HuggingFace Inference API you must have a HF account, the API verifies this by checking your account's token. This token will not be used for anything else.
* **Benchmark context files:** In order to solve the benchmark tasks the agent will need to reference documentation and analyze data which is spread out across multiple files, just like a real Data Analyst would.

### HuggingFace Token Setup

In [43]:
import time
import os
import json
import re
import datasets
import pandas as pd
from smolagents import CodeAgent
from smolagents.agents import ActionStep
from smolagents.models import OpenAIServerModel
from huggingface_hub import hf_hub_download

# Load OpenRouter API key from secrets
openrouter_key = None
openrouter_path = os.path.abspath(os.path.join(os.getcwd(), "..", "secrets", "openrouter_credentials.txt"))
if os.path.exists(openrouter_path):
    with open(openrouter_path, "r") as f:
        content = f.read()
    # Extract API key - look for pattern like sk-or-v1-...
    key_match = re.search(r"sk-or-v1-[A-Za-z0-9]+", content)
    if key_match:
        openrouter_key = key_match.group(0)
        print("✓ Loaded OpenRouter API key from secrets")
    else:
        print("⚠️ Warning: No valid OpenRouter key found in", openrouter_path)
else:
    print("⚠️ Error: OpenRouter credentials file not found at", openrouter_path)
    raise FileNotFoundError("OpenRouter credentials required to run this notebook")

# Try to import DABstep utilities
try:
    from dabstep_benchmark.utils import evaluate
    DABSTEP_AVAILABLE = True
except ImportError:
    print("Warning: DABstep benchmark utilities not available.")
    DABSTEP_AVAILABLE = False
    def evaluate(*args, **kwargs):
        raise RuntimeError("DABstep utilities not installed.")

notebook_start_time = time.time()


✓ Loaded OpenRouter API key from secrets


#### Download context files
First we download the context files from the [Benchmark's Dataset](https://huggingface.co/datasets/adyen/DABstep) so that our agent can access them.


In [44]:
CONTEXT_FILENAMES = [
    "data/context/acquirer_countries.csv",
    "data/context/payments-readme.md",
    "data/context/payments.csv",
    "data/context/merchant_category_codes.csv",
    "data/context/fees.json",
    "data/context/merchant_data.json",
    "data/context/manual.md",
]
# Store data locally in the repo, not in /tmp
DATA_DIR = os.path.join(os.path.dirname(os.path.abspath(".")), "data", "context_files")
os.makedirs(DATA_DIR, exist_ok=True)

for filename in CONTEXT_FILENAMES:
    hf_hub_download(
        repo_id="adyen/DABstep",
        repo_type="dataset",
        filename=filename,
        local_dir=DATA_DIR,
        force_download=False
    )

CONTEXT_FILENAMES = [f"{DATA_DIR}/{filename}" for filename in CONTEXT_FILENAMES]

for file in CONTEXT_FILENAMES:
    if os.path.exists(file):
        print(f"{file} exists.")
    else:
        print(f"{file} does not exist.")

/home/mykola/repos/dabstep_test/data/context_files/data/context/acquirer_countries.csv exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/payments-readme.md exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/payments.csv exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_category_codes.csv exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/fees.json exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_data.json exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/manual.md exists.


## Agent

Here we will setup a simple zero-shot prompt for the agent. It has two parts, the general prompt and then a quick outline of which files are available.

In [45]:
# Available OpenRouter models
MODELS_OPENROUTER = {
    "qwen2.5": "qwen/qwen-2.5-coder-32b-instruct",
    "qwen-coder-480b": "qwen/qwen3-coder",
    "deepseek-v3.1": "deepseek/deepseek-chat-v3.1",
    "deepseek-v3-terminus": "deepseek/deepseek-v3.1-terminus",
    "deepseek-v3.2": "deepseek/deepseek-v3.2-exp",
    "kimi-k2-0905": "moonshotai/kimi-k2-0905",
    "glm-4.5-air": "z-ai/glm-4.5-air",
    "glm-4.5": "z-ai/glm-4.5",
    "gpt-oss-20b": "openai/gpt-oss-20b",
    "gpt-oss-120b": "openai/gpt-oss-120b",
    "gpt-5-mini": "openai/gpt-4o-mini",
    "gpt-5-nano": "openai/gpt-4o-mini",
    "grok-code-fast1": "x-ai/grok-code-fast-1",
    "grok-4-fast": "x-ai/grok-4-fast",
}

# Select which model to use
MODEL_KEY = "qwen-coder-480b"
MODEL_ID = MODELS_OPENROUTER[MODEL_KEY]

model = OpenAIServerModel(
    model_id=MODEL_ID,
    api_base="https://openrouter.ai/api/v1",
    api_key=openrouter_key
)
print(f"✓ Using OpenRouter model: {MODEL_ID}")


✓ Using OpenRouter model: qwen/qwen3-coder


## 3.4 Testing Agent

In [46]:
# Use the model instance created in the previous cell
MAX_STEPS = 7

agent = CodeAgent(
    tools=[],
    model=model,  # Use the model instance from previous cell
    additional_authorized_imports=["numpy", "pandas", "json", "csv", "os", "glob", "markdown"],
    max_steps=MAX_STEPS,
    verbosity_level=3,
)

In [47]:
PROMPT = """You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.
You have these files available:
{context_files}
Don't forget to reference any documentation in the data dir before answering a question.

Here is the question you need to answer:
{question}

Here are the guidelines you must follow when answering the question above:
{guidelines}
"""
question = "What are the unique set of merchants in the payments data?"
guidelines = "Answer with a comma separated list"

PROMPT = PROMPT.format(
    context_files=CONTEXT_FILENAMES,
    question=question,
    guidelines=guidelines
)

answer = agent.run(PROMPT)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert data analyst and you will answer factoid questions by loading and referencing the             │
│ files/documents listed below.                                                                                   │
│ You have these files available:                                                                                 │
│ ['/home/mykola/repos/dabstep_test/data/context_files/data/context/acquirer_countries.csv',                      │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/payments-readme.md',                           │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/payments.csv',                                 │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_category_codes.csv',                  │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/fees.json',                                    │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_data.json',                           │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/manual.md'\]                                   │
│ Don't forget to reference any documentation in the data dir before answering a question.                        │
│                                                                                                                 │
│ Here is the question you need to answer:                                                                        │
│ What are the unique set of merchants in the payments data?                                                      │
│                                                                                                                 │
│ Here are the guidelines you must follow when answering the question above:                                      │
│ Answer with a comma separated list                                                                              │
│                                                                                                                 │
╰─ OpenAIServerModel - qwen/qwen3-coder ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<code>                                                                                                             
import pandas as pd                                                                                                
                                                                                                                   
# Load the payments data                                                                                           
payments_df = pd.read_csv('/home/mykola/repos/dabstep_test/data/context_files/data/context/payments.csv')          
                                                                                                                   
# Get unique merchants                                                                                             
unique_merchants = payments_df['merchant_name'].unique()                                                           
                                                                                                                   
# Convert to a comma-separated string                                                                              
result = ', '.join(unique_merchants)                                                                               
                                                                                                                   
print(result)                                                                                                      
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Load the payments data                                                                                         
  payments_df = pd.read_csv('/home/mykola/repos/dabstep_test/data/context_files/data/context/payments.csv')        
                                                                                                                   
  # Get unique merchants                                                                                           
  unique_merchants = payments_df['merchant_name'].unique()                                                         
                                                                                                                   
  # Convert to a comma-separated string                                                                            
  result = ', '.join(unique_merchants)                                                                             
                                                                                                                   
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'unique_merchants = payments_df['merchant_name'\].unique()' due to: InterpreterError:
Could not index         psp_reference               merchant   card_scheme  year  hour_of_day  \
0         20034594130         Crossfit_Hanna        NexPay  2023           16   
1         36926127356         Crossfit_Hanna        NexPay  2023           23   
2         31114608278  Belles_cookbook_store    GlobalCard  2023            4   
3         68442235288         Crossfit_Hanna        NexPay  2023            3   
4         81404384199         Crossfit_Hanna        NexPay  2023           17   
...               ...                    ...           ...   ...          ...   
138231    89124881850         Crossfit_Hanna    GlobalCard  2023           20   
138232    31688111629         Crossfit_Hanna    GlobalCard  2023            4   
138233    54380760510         Crossfit_Hanna  TransactPlus  2023            9   
138234    61488790815         Crossfit_Hanna    GlobalCard  2023           13   
138235    27351484054         Crossfit_Hanna        NexPay  2023           15   

        minute_of_hour  day_of_year  is_credit  eur_amount ip_country  ...  \
0                   21           12      False      151.74         SE  ...   
1                   58           75      False       45.70         NL  ...   
2                   30           96      False       14.11         NL  ...   
3                    5           77       True      238.42         LU  ...   
4                   30           83      False       67.13         NL  ...   
...                ...          ...        ...         ...        ...  ...   
138231              10          223       True      246.48         NL  ...   
138232              48          232      False       85.74         ES  ...   
138233              17          180      False       39.30         FR  ...   
138234              20          127       True      116.97         LU  ...   
138235              31          348       True       43.08         BE  ...   

       device_type              ip_address           email_address  \
0          Windows  pKPYzJqqwB8TdpY0jiAeQw  0AKXyaTjW7H4m1hOWmOKBQ   
1            Linux  uzUknOkIqExYsWv4X14GUg  _Gm8at1k2ojYAM_wSEptNw   
2            MacOS  3VO1v_RndDg6jzEiPjfvoQ                     NaN   
3              iOS  3qbuXGoFldniCC6r1X8K0Q  5VW_2O6ku_0p_fLLwuC1vw   
4          Windows  9WMJJdgtop6jkkyerxMvuQ  Alb1iUIxIqlW8YUeYVGTzg   
...            ...                     ...                     ...   
138231     Windows  MrIIK0HAweKXa2VLvMVLwA  RzxrSatGNna_3JbQGBiMhA   
138232       Other                     NaN  bkK9YEMg26xB8Y--u_niTg   
138233       Other  8wSmQINl-EUZkKetaoiFKw  sOv88acnUF0uGCOEh4z05A   
138234       MacOS  Bv7KNKMi1yJ2HEHXRTUriA  pfsVN33UDQS-7n6yIKhGFQ   
138235     Android                     NaN  F57Ru15Epqan6P08UXULWg   

                   card_number shopper_interaction card_bin  \
0       uRofX46FuLUrSOTz8AW5UQ           Ecommerce     4802   
1       6vqQ89zfCeFk6s4VOoWZFQ           Ecommerce     4920   
2       EmxSN8-GXQw3RG_2v7xKxQ           Ecommerce     4571   
3       wG2VTvj2TfVG-NRDzifMHw           Ecommerce     4017   
4       0khzuCj7aQ1e51S5vWR8gg           Ecommerce     4532   
...                        ...                 ...      ...   
138231  ghNPy4XwuCbTFEIO9tN0yg           Ecommerce     4556   
138232  Kt5dqC34pAtCd1BEi6db1A           Ecommerce     4236   
138233  -JH1K3c8mT5P2i_3eDgC7g           Ecommerce     4622   
138234  0wWz3wrceqPFf7B1rRcpJg           Ecommerce     4622   
138235  G8nqqAGM5vAYmMwTYiFDFw           Ecommerce     4920   

        has_fraudulent_dispute  is_refused_by_adyen  aci acquirer_country  
0                        False                False    F               NL  
1                        False                False    F               NL  
2                        False                False    F               US  
3                        False                False    D

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<code>                                                                                                             
final_answer("Crossfit_Hanna, Belles_cookbook_store, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, Rafa_AI")     
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Crossfit_Hanna, Belles_cookbook_store, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, Rafa_AI")   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Crossfit_Hanna, Belles_cookbook_store, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, Rafa_AI

[Step 4: Duration 1.90 seconds| Input tokens: 15,527 | Output tokens: 269]

In [48]:
# You can inspect the steps taken by the agent by doing this
def clean_reasoning_trace(trace: list) -> list:
  for step in trace:
      # Remove memory from logs to make them more compact.
      if hasattr(step, "memory"):
          step.memory = None
      if isinstance(step, ActionStep):
          step.agent_memory = None
  return trace

# Access agent's reasoning trace
if hasattr(agent, 'memory') and hasattr(agent.memory, 'steps'):
    steps = agent.memory.steps
    for step in clean_reasoning_trace(steps):
        print(step)
else:
    print("Agent steps not available")

TaskStep(task="You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.\nYou have these files available:\n['/home/mykola/repos/dabstep_test/data/context_files/data/context/acquirer_countries.csv', '/home/mykola/repos/dabstep_test/data/context_files/data/context/payments-readme.md', '/home/mykola/repos/dabstep_test/data/context_files/data/context/payments.csv', '/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_category_codes.csv', '/home/mykola/repos/dabstep_test/data/context_files/data/context/fees.json', '/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_data.json', '/home/mykola/repos/dabstep_test/data/context_files/data/context/manual.md']\nDon't forget to reference any documentation in the data dir before answering a question.\n\nHere is the question you need to answer:\nWhat are the unique set of merchants in the payments data?\n\nHere are the guidelines you must

In [49]:
notebook_end_time = time.time()
print(f"Notebook runtime: {notebook_end_time - notebook_start_time:.2f} seconds")

Notebook runtime: 16.66 seconds
